In [1]:
import numpy as np
import math
import pandas as pd
import threading
import time
import queue

In [2]:
def mapper(order):
    sourcefile = './data/source0' + str(order)
    map_filename = './mapper/mapper0' + str(order)
    with open(map_filename, 'w') as f:
        for line in open(sourcefile):
            line_word_list = line.strip().split(',')
            for word in line_word_list:
                f.write(word.strip() + '\t1\n')

In [3]:
thread_list = []
for i in range(1, 10):
    thread_list.append(threading.Thread(target=mapper, args=(i, )))

In [4]:
for tr in thread_list:
    tr.start()

In [8]:
def combiner(order):
    sourcefile = './mapper/mapper0' + str(order)
    dict = {}
    for line in open(sourcefile):
        element = line.split('\t')
        word, num = element[0], (int)(element[1])
        if word in dict:
            dict[word] += 1
        else:
            dict.update({word : 1})
    return dict

In [9]:
class CombineThread(threading.Thread):
    def __init__(self, func, args=()):
        super(CombineThread, self).__init__()
        self.func = func
        self.args = args
    
    def run(self):
        self.res = self.func(*self.args)

    def get_result(self):
        try:
            return self.res
        except Exception as e:
            return None

In [10]:
combine_dict_list, threads = [], []
for i in range(1, 10):
    tr = CombineThread(func=combiner, args=(i, ))
    tr.start()
    threads.append(tr)

for tr in threads:
    tr.join()
    combine_dict_list.append(tr.get_result())

In [19]:
def shuffle():
    filename = './shuffle/shuffle0'
    w1 = open(filename + '1', 'w')
    w2 = open(filename + '2', 'w')
    w3 = open(filename + '3', 'w')
    for dic in combine_dict_list:
        for key, value in dic.items():
            if key[0].isupper() or (key[0] >= 'a' and key[0] <= 'c') or (not key[0].isalpha()):
                w1.write(key + '\t' + str(value) + '\n')
            elif key[0] >= 'd' and key[0] <= 'o':
                w2.write(key + '\t' + str(value) + '\n')
            else:
                w3.write(key + '\t' + str(value) + '\n')
    w1.close()
    w2.close()
    w3.close()

In [20]:
shuffle()

In [28]:
def Reducer(order):
    sourcefile = './shuffle/shuffle0' + (str)(order)
    dict = {}
    for line in open(sourcefile):
        element = line.split('\t')
        word, num = element[0], (int)(element[1])
        if word in dict:
            dict[word] += num
        else:
            dict.update({word : num})
    dict = sorted(dict.items(), key=lambda d: d[0])
    filename = './reducer/reducer0' + (str)(order)
    w = open(filename, 'w')
    for ind in range(len(dict)):
        w.write(dict[ind][0] + '\t' + str(dict[ind][1]) + '\n')
    w.close()

In [29]:
reducer_thread_list = []
for i in range(1, 4):
    reducer_thread_list.append(threading.Thread(target=Reducer, args=(i, )))

In [30]:
for tr in reducer_thread_list:
    tr.start()

In [31]:
sourcefile = './reducer/reducer0'
w = open('ans', 'w')
for i in range(1, 4):
    wt = open(sourcefile + str(i))
    line = wt.readline()
    while line:
        w.write(line)
        line = wt.readline()